# Brain Tumor Analysis

In this project we are going to predict wether if it is a **Brain Tumor** or not, using **Magnetic Resonance Imaging** *(MRI)*, from Kaggle.


## Getting Dataset
Here we extract our data and give the right label to each image.

In [11]:
import os
import numpy as np
import pandas as pd

paht_yes = "brain_tumor_datasets/yes"
path_no = "brain_tumor_datasets/no"

data = []

def extract_images(folder, label):
    for filename in os.listdir(folder):
        filepath = os.path.join(folder, filename)
        if os.path.isfile(filepath):
            data.append({"image": filepath, "label": label})

extract_images(paht_yes, label=1)
extract_images(path_no, label=0)

df = pd.DataFrame(data)

df

,image,label
0,brain_tumor_datasets/yes\Y1.jpg,1
1,brain_tumor_datasets/yes\Y10.jpg,1
2,brain_tumor_datasets/yes\Y100.JPG,1
3,brain_tumor_datasets/yes\Y101.jpg,1
4,brain_tumor_datasets/yes\Y102.jpg,1
...,...,...
248,brain_tumor_datasets/no\No18.jpg,0
249,brain_tumor_datasets/no\No19.jpg,0
250,brain_tumor_datasets/no\No20.jpg,0
251,brain_tumor_datasets/no\No21.jpg,0
